<a href="https://colab.research.google.com/github/jcmachicao/pucp__gobierno_sistemas_proyectos/blob/main/pucp_men623__benefit_value_montecarlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Benefit Value Cost Simulation: Bicycle Case
José Carlos Machicao

In [1]:
import random
import pandas as pd

In [2]:
scenarios = {
    "sport_priority": {
        "Smooth gear system": 0.9,
        "Reliable brakes": 0.7,
        "Lightweight frame": 1.0,
        "Aerodynamics": 0.9,
        "Proper fit and comfort": 0.6,
        "Night visibility system": 0.3,
        "Tire grip and stability": 0.5,
        "Maintenance ease": 0.4
    },
    "high_safety": {
        "Smooth gear system": 0.6,
        "Reliable brakes": 1.0,
        "Lightweight frame": 0.4,
        "Aerodynamics": 0.3,
        "Proper fit and comfort": 0.9,
        "Night visibility system": 0.95,
        "Tire grip and stability": 0.9,
        "Maintenance ease": 0.8
    }
}

In [3]:
benefit_profiles = {
    "safety": {
        "Reliable brakes": 1.0,
        "Night visibility system": 0.95,
        "Tire grip and stability": 0.9,
        "Proper fit and comfort": 0.8,
        "Smooth gear system": 0.6
    },
    "sport_excellence": {
        "Lightweight frame": 1.0,
        "Aerodynamics": 0.9,
        "Smooth gear system": 0.85,
        "Proper fit and comfort": 0.7
    },
    "bike_maintenance": {
        "Maintenance ease": 1.0,
        "Smooth gear system": 0.5,
        "Reliable brakes": 0.4
    }
}

feature_costs = {
    "Smooth gear system": 300,
    "Reliable brakes": 250,
    "Lightweight frame": 500,
    "Aerodynamics": 350,
    "Proper fit and comfort": 200,
    "Night visibility system": 150,
    "Tire grip and stability": 180,
    "Maintenance ease": 100
}

In [4]:
def forecast_benefit(current_features: dict, benefit_profile: dict) -> float:
    numerator = 0.0
    denominator = 0.0
    for feature, weight in current_features.items():
        benefit_weight = benefit_profile.get(feature, 0.0)
        numerator += weight * benefit_weight
        denominator += weight
    return numerator / denominator if denominator > 0 else 0.0

def forecast_all(current_features: dict) -> dict:
    return {
        name: forecast_benefit(current_features, profile)
        for name, profile in benefit_profiles.items()
    }

def simulate_cost(current_features: dict) -> float:
    total_cost = 0.0
    for feature, weight in current_features.items():
        unit_cost = feature_costs.get(feature, 0.0)
        total_cost += unit_cost * weight
    management_overhead = 200
    return total_cost + management_overhead

def compute_ratios(forecast: dict, total_cost: float, current_features: dict) -> dict:
    total_scope = sum(current_features.values())
    return {
        f"{name}_per_cost": (score / total_cost if total_cost else 0)
        for name, score in forecast.items()
    } | {"scope_per_cost": total_scope / total_cost if total_cost else 0}

def monte_carlo_simulation(n_iterations: int, target: dict) -> pd.DataFrame:
    feature_names = list(feature_costs.keys())
    simulations = []
    for _ in range(n_iterations):
        random_features = {f: round(random.uniform(0, 1), 2) for f in feature_names}
        forecast = forecast_all(random_features)
        cost = simulate_cost(random_features)
        ratios = compute_ratios(forecast, cost, random_features)
        error = sum((ratios.get(f"{k}_per_cost", 0) - v) ** 2 for k, v in target.items())
        row = {**random_features, **ratios, "error": error}
        simulations.append(row)
    df = pd.DataFrame(simulations)
    return df.sort_values(by="error").head(3)

In [5]:
# Example usage:
selected_scenario = "sport_priority"
features = scenarios[selected_scenario]

print(f"Feature values for {selected_scenario} scenario:")
for feature, value in features.items():
    print(f"{feature}: {value}")

forecast = forecast_all(features)
print("\nForecasted Systemic Benefits:")
for name, score in forecast.items():
    print(f"{name.replace('_', ' ').title()}: {score:.2f}")

total_cost = simulate_cost(features)
print(f"\nSimulated Total Cost: ${total_cost:.2f}")

ratios = compute_ratios(forecast, total_cost, features)
print("\nBenefit/Cost and Scope/Cost Ratios:")
for name, ratio in ratios.items():
    print(f"{name.replace('_', ' ').title()}: {ratio:.4f}")

Feature values for sport_priority scenario:
Smooth gear system: 0.9
Reliable brakes: 0.7
Lightweight frame: 1.0
Aerodynamics: 0.9
Proper fit and comfort: 0.6
Night visibility system: 0.3
Tire grip and stability: 0.5
Maintenance ease: 0.4

Forecasted Systemic Benefits:
Safety: 0.46
Sport Excellence: 0.57
Bike Maintenance: 0.21

Simulated Total Cost: $1755.00

Benefit/Cost and Scope/Cost Ratios:
Safety Per Cost: 0.0003
Sport Excellence Per Cost: 0.0003
Bike Maintenance Per Cost: 0.0001
Scope Per Cost: 0.0030


In [6]:
# Monte Carlo target and optimization
initial_target = {
    "safety": 0.005,
    "sport_excellence": 0.006,
    "bike_maintenance": 0.004
}

In [7]:
print("\nRunning Monte Carlo Optimization...")
top_matches = monte_carlo_simulation(500, initial_target)
print("\nTop 3 configurations matching target ratios:")
top_matches


Running Monte Carlo Optimization...

Top 3 configurations matching target ratios:


,Smooth gear system,Reliable brakes,Lightweight frame,Aerodynamics,Proper fit and comfort,Night visibility system,Tire grip and stability,Maintenance ease,safety_per_cost,sport_excellence_per_cost,bike_maintenance_per_cost,scope_per_cost,error
286,0.31,0.20,0.33,0.02,0.04,0.20,0.01,0.08,0.000921,0.000955,0.000470,0.002114,0.000055
88,0.17,0.08,0.01,0.15,0.10,0.14,0.71,0.49,0.001023,0.000356,0.000601,0.003386,0.000059
35,0.32,0.10,0.18,0.31,0.30,0.36,0.26,0.25,0.000755,0.000641,0.000307,0.002949,0.000060
